In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing, SimpleExpSmoothing
from tqdm import tqdm

In [16]:
#x_train = pd.read_pickle('x_train.pkl')
x_test = pd.read_pickle("x_test.pkl")
y_train = pd.read_pickle('y_train.pkl')
y_test = pd.read_pickle("y_test.pkl")

In [3]:
x_test = x_test.iloc[:,12:]
x_test

,sales,x_1DaysAgo,x_2DaysAgo,x_3DaysAgo,x_4DaysAgo,x_5DaysAgo,x_6DaysAgo,x_7DaysAgo,x_8DaysAgo,x_9DaysAgo,...,x_51DaysAgo,x_52DaysAgo,x_53DaysAgo,x_54DaysAgo,x_55DaysAgo,x_56DaysAgo,x_57DaysAgo,x_58DaysAgo,x_59DaysAgo,x_60DaysAgo
1912,6,16.0,2.0,6.0,3.0,5.0,7.0,30.0,4.0,4.0,...,3.0,8.0,5.0,6.0,3.0,6.0,5.0,3.0,1.0,1.0
1913,4,6.0,16.0,2.0,6.0,3.0,5.0,7.0,30.0,4.0,...,8.0,3.0,8.0,5.0,6.0,3.0,6.0,5.0,3.0,1.0
1914,5,4.0,6.0,16.0,2.0,6.0,3.0,5.0,7.0,30.0,...,6.0,8.0,3.0,8.0,5.0,6.0,3.0,6.0,5.0,3.0
1915,7,5.0,4.0,6.0,16.0,2.0,6.0,3.0,5.0,7.0,...,9.0,6.0,8.0,3.0,8.0,5.0,6.0,3.0,6.0,5.0
1916,1,7.0,5.0,4.0,6.0,16.0,2.0,6.0,3.0,5.0,...,3.0,9.0,6.0,8.0,3.0,8.0,5.0,6.0,3.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19404,4,5.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0,1.0,...,5.0,0.0,2.0,3.0,5.0,0.0,2.0,4.0,1.0,0.0
19405,0,4.0,5.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0,...,7.0,5.0,0.0,2.0,3.0,5.0,0.0,2.0,4.0,1.0
19406,2,0.0,4.0,5.0,0.0,2.0,1.0,0.0,0.0,0.0,...,3.0,7.0,5.0,0.0,2.0,3.0,5.0,0.0,2.0,4.0
19407,2,2.0,0.0,4.0,5.0,0.0,2.0,1.0,0.0,0.0,...,1.0,3.0,7.0,5.0,0.0,2.0,3.0,5.0,0.0,2.0


In [4]:
np.shape(np.array(x_test))

(563304, 61)

In [5]:
forecast_horizon = 1
es_base = []

In [6]:
# train the ES model:
import warnings
warnings.filterwarnings("ignore", category=Warning)

In [7]:

for i in tqdm(range(len(x_test))):
    ES = SimpleExpSmoothing(x_test.iloc[i,:])
    result = ES.fit()
    forc = result.forecast(steps=forecast_horizon)
    es_base.append(forc)
    #tru = y_test[i]
    #es_mase.append(forecast_mase(tru,forc,1))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 563304/563304 [34:57<00:00, 268.61it/s]


In [8]:
np.shape(es_base)

(563304, 1)

In [10]:
es_b = np.array(es_base).reshape(1437,392)
tru = np.array(y_test).reshape(1437,392)

In [11]:
tru

array([[ 4.,  5.,  7., ...,  0.,  0.,  0.],
       [ 9.,  4.,  3., ...,  2.,  1.,  0.],
       [ 7., 13.,  9., ...,  0.,  0.,  1.],
       ...,
       [11.,  9.,  4., ...,  1.,  0.,  2.],
       [13., 13., 16., ...,  1.,  1.,  0.],
       [42., 26., 13., ...,  2.,  5.,  1.]])

In [12]:
def forecast_mase(truth_1,forecast_1, epsilon):
    from sklearn.metrics import mean_absolute_error
    mase = []
    forecast = np.array(forecast_1)
    truth = np.array(truth_1)
    tru_diff = np.abs(truth[:, 1:] - truth[:, :-1])  # achieve .diff()
    mean_diff = np.mean(tru_diff, axis=1)
    for i in range(len(forecast)):
        mae_ = mean_absolute_error( truth[i, :],forecast[i, :])
        mean_diff_i = mean_diff[i] if mean_diff[i] != 0 else epsilon
        mase.append(mae_ / mean_diff_i)
    return mase

In [13]:
'''
##### base forecasts:
re_base_all = []
re_base_216 = []
for i in range(len(es_b)):
    pred = es_b[i].reshape(14,28)
    truth = tru[i].reshape(14,28)
    re_base_all.append(forecast_mase(truth, pred, 1))
    re_base_216.append(np.mean(re_base_all))'''

'\n##### base forecasts:\nre_base_all = []\nre_base_216 = []\nfor i in range(len(es_b)):\n    pred = es_b[i].reshape(14,28)\n    truth = tru[i].reshape(14,28)\n    re_base_all.append(forecast_mase(truth, pred, 1))\n    re_base_216.append(np.mean(re_base_all))'

In [14]:
'''
np.mean(re_base_216)'''

'\nnp.mean(re_base_216)'

In [17]:
#### Summing Matrix
l1 = np.ones(10).reshape(1,10)
l2 = [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 1, 1, 1]]
l3 = np.eye(10)
S_Matrix = np.concatenate((l1, l2, l3), axis=0)
################# TD method:
# G matrix for TD, proportions of historical averages
def G_Matrix_TD(df ,ts_length, ts_niah, ts_nibl):
    n = int(len(df)/(ts_length*ts_niah))
    G_td = []
    df_ = np.array(df).reshape(n,int(len(df)/n))
    for i in tqdm(range(n)):
        ts = df_[i].reshape(ts_niah,ts_length)
        prop = (np.mean(ts[-10:,:],axis = 1)/np.mean(ts[0,:])).reshape(ts_nibl,1)
        zero = np.zeros((ts_nibl,ts_niah-1))
        g_td = np.concatenate((prop, zero), axis=1)
        G_td.append(g_td)
    return G_td
# Different G matrix for 3 methods:
################# BU method:
# G matrix for BU
g1 = np.zeros((10,4))#（ bottom_level , except_bottom level）
g2 = np.eye(10)# bottom level
G_bu = np.concatenate((g1,g2), axis=1)

################# TD method:
# parameters: bottom level data, top level data, time series length, 
G_td = G_Matrix_TD(y_train, 1852, 14, 10)

################# MinT method:
I = np.ones(10)
kh= 1
W = np.diag(S_Matrix@I)
G_mint = np.linalg.inv(S_Matrix.T@np.linalg.inv(W)@S_Matrix)@S_Matrix.T@np.linalg.inv(W)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1437/1437 [00:00<00:00, 18395.28it/s]


In [18]:
################## Different SG matrix for 3 methods:
SG_bu = np.dot(S_Matrix, G_bu)
SG_mint = np.dot(S_Matrix,G_mint)
SG_td = []
for i in tqdm(range(len(G_td))):
    sg_td = S_Matrix@G_td[i]
    SG_td.append(sg_td)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1437/1437 [00:00<00:00, 358698.74it/s]


In [19]:
##### base forecasts:
re_base_all = []
re_base_216 = []
for i in range(len(es_b)):
    pred = es_b[i].reshape(14,28)
    truth = tru[i].reshape(14,28)
    re_base_all.append(forecast_mase(truth, pred, 1))
    re_base_216.append(np.mean(re_base_all))
##### bu forecasts:
re_bu_all = []
re_bu_216 = []
for i in range(len(es_b)):
    pred = es_b[i].reshape(14,28)
    truth = tru[i].reshape(14,28)
    y_coh_frec_bu = np.dot(SG_bu, pred)
    re_bu_all.append(forecast_mase(truth, y_coh_frec_bu, 1))
    re_bu_216.append(np.mean(re_bu_all))
##### td forecasts:
re_td_all = []
re_td_216 = []
for i in range(len(es_b)):
    pred = es_b[i].reshape(14,28)
    truth = tru[i].reshape(14,28)
    y_coh_frec_td = np.dot(SG_td[i], pred)
    re_td_all.append(forecast_mase(truth,y_coh_frec_td, 1))
    re_td_216.append(np.mean(re_td_all))
##### mint forecasts:
re_mint_all = []
re_mint_216 = []
for i in range(len(es_b)):
    pred = es_b[i].reshape(14,28)
    truth = tru[i].reshape(14,28)
    y_coh_frec_mint = np.dot(SG_mint, pred)
    re_mint_all.append(forecast_mase(truth, y_coh_frec_mint, 1))
    re_mint_216.append(np.mean(re_mint_all))

In [20]:
print("ES's forecasts:", np.mean(re_base_216))
print("ES with BU's forecasts:", np.mean(re_bu_216))
print("ES with TD's forecasts:", np.mean(re_td_216))
print("ES with MinT's forecasts:", np.mean(re_mint_216))

ES's forecasts: 1.0846238003881108
ES with BU's forecasts: 1.0792911014546953
ES with TD's forecasts: 1.1493052015204683
ES with MinT's forecasts: 1.0826059449237881
